In [11]:
import os
import pickle
import numpy as np

from our_Wolpertinger import Wolpertinger
from TD3 import TD3
from tqdm import tqdm

import torch

import sys
sys.path.append(os.path.join(os.getcwd(), 'Simulator'))
from WordCounting import WordCountingEnv

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
with open(f"./offline/random_con_2.pkl", 'rb') as f:
    buffer = pickle.load(f)

In [4]:
buffer.sample(1)

(tensor([[0.0479, 0.1848, 0.0139, 0.1031, 0.1681, 0.1634, 0.1469, 0.0886, 0.0550,
          0.0282, 0.0495, 0.1870, 0.1697, 0.0109, 0.0841, 0.1191, 0.1008, 0.1460,
          0.1028, 0.0301, 0.0190, 0.1544, 0.0960, 0.0432, 0.0964, 0.1680, 0.0388,
          0.0435, 0.1546, 0.1860, 4.9282, 4.9646, 5.0319, 4.9083, 5.1755, 4.8168,
          5.0465, 5.0217, 5.0971, 4.9819, 5.1944, 4.9368, 5.1557, 4.9731, 4.9883,
          4.8844, 5.0350, 4.8795, 4.7993, 5.0895]]),
 tensor([[0.6597, 0.5404, 0.3842, 0.3095, 0.3700, 0.4329, 0.9100, 0.4347, 0.3120,
          0.4218, 0.8331, 0.2503, 0.1493, 0.3705, 0.5189, 0.6914, 0.4966, 0.7308,
          0.5290, 0.9200, 0.9440, 0.1746, 0.9597, 0.4424, 0.9960, 0.7969, 0.4056,
          0.3787, 0.8597, 0.1899]]),
 tensor([[0.1382, 0.1132, 0.0805, 0.0648, 0.0775, 0.0907, 0.1906, 0.0910, 0.0653,
          0.0883, 0.1517, 0.0456, 0.0272, 0.0675, 0.0945, 0.1259, 0.0905, 0.1331,
          0.0963, 0.1676, 0.1536, 0.0284, 0.1561, 0.0720, 0.1620, 0.1296, 0.0660,
        

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
buffer.device = device

In [6]:
s = buffer.sample(1)
state_dim = s[0].shape[-1]
action_dim = s[1].shape[-1]

In [26]:
def eval_policy(policy, eval_env, eval_steps=1):
    avg_reward = 0.
    state, done = eval_env.reset(), False
    for _ in range(eval_steps):
        action = policy.select_action(np.array(state))
        state, reward, done, _ = eval_env.step(action)
        avg_reward += reward
        print()

    avg_reward /= eval_steps

    print("---------------------------------------")
    print(f"Evaluation over {eval_steps} steps:{avg_reward:.3f}")
    print("---------------------------------------")
    return avg_reward

In [29]:
model = TD3(state_dim, action_dim, 1)

In [44]:
for i in tqdm(range(500)):
    model.train(buffer, batch_size=128)
    # if i % 2000 == 0 and i >= 2000:
    #     eval_results.append(eval_policy(model, eval_env))

100%|██████████| 500/500 [00:06<00:00, 76.95it/s]


In [45]:
eval_env = WordCountingEnv(seed=2)

Building Topology
spout 20
WordCount 40
Database 40


In [46]:
eval_policy(model, eval_env, 2)

---------------------------------------
Evaluation over 2 steps:-8.571
---------------------------------------


-8.57144549517184

# BCQ

In [4]:
from bcq import BCQ
offline_model = BCQ(state_dim, action_dim, 1, device)

NameError: name 'state_dim' is not defined

In [86]:
offline_model.train(buffer, 5000)
print('finish training')
# eval_results.append(eval_policy(offline_model, eval_env))

finish training


In [91]:
np.argmax(offline_model.select_action(buffer.next_state[9749]).reshape((-1,5)), axis=1)

array([0, 0, 3, 3, 4, 2, 2, 3, 3, 0, 0, 4, 0, 4, 4, 0, 3, 2, 3, 0])

In [98]:
offline_model.critic(torch.FloatTensor([buffer.next_state[9749]]), torch.FloatTensor([offline_model.select_action(buffer.next_state[9749])]))

(tensor([[-10179.7705]], grad_fn=<AddmmBackward>),
 tensor([[-10130.4355]], grad_fn=<AddmmBackward>))